In [15]:
import speech_recognition as sr
import time

%run speak.ipynb import speak
%run chatgpt.ipynb
%run notion.ipynb import create_notion_page

In [16]:
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Calibrating...")
        r.adjust_for_ambient_noise(source, duration=5)

        print("Done!")
        interaction = True
        while interaction:
            text = ""
            print("Listening...")
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=30)
                print("Recognizing...")
                
                # whisper model options are found here: 
                # https://github.com/openai/whisper#available-models-and-languages
                text = r.recognize_whisper(
                    audio,
                    model="medium",
                    show_dict=True,
                )["text"]

                speak("What I understood is: ")
                time.sleep(2)
                speak(text)

                interaction = False

            except Exception as e:
                print(f"I'm sorry, I didn't understand you. The error was: {e}s")

            try:
                response_text = chatgpt_chain.predict(human_input=text)
                response_text = response_text.strip()

                if response_text:
                    time.sleep(1)
                    print(response_text)
                    speak(response_text)

                    create_notion_page(text, response_text)

            except Exception as e:
                print(f"Sorry I couldn't answer. The error was: {e}s")